In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
label = []  
path = []  
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'  
for dir_name, _, filenames in os.walk(fish_dir):  
 for filename in filenames:  
     if os.path.splitext(filename)[-1] == '.png': 
         if dir_name.split()[-1]!='GT':  # finds the "GT" in the directory names from reverse 
             label.append(os.path.split(dir_name)[-1])  
             path.append(os.path.join(dir_name, filename))

data=pd.DataFrame(columns=['path','label'])
data['path'] = path
data['label'] = label
print(data.shape) 
print(data['label'].unique()) #class number
#print(label,'/n')
#print(path[:3])

This code block parses the images and labels from the datasets file paths for use in later stages. Then, it saves them in 2 separate arrays named path[] and label[].
The first output shows that there are 9000 data in the dataset and 2 columns: path, label. The second output shows 9 different classes to be used in the classification.

In [ ]:
import os
print(os.listdir('/kaggle/input'))
from PIL import Image
import os

image_directory = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# used os.walk to check images in subfolders too
for root, dirs, files in os.walk(image_directory):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_path = os.path.join(root, filename)
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
                    print(f'{filename}: {width}x{height}')
            except Exception as e:
                print(f'The image could not be read: {filename}, Error: {e}')


**This code block was written to set the dimensions of the images to be used in model training.**
The file extensions of the images in the dataset are checked using the file path, then the dimensions of the images are determined with the size() function and printed to the screen. After checking the dimensions, the appropriate X*X size to be given to the model is decided.


In [ ]:
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

dataset_path = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

images = []
labels = []

# determine the image dimensions
img_width, img_height = 128, 128
            
# looped subfolders in the main folder for each fish species
for fish_type in os.listdir(dataset_path):
    fish_type_path = os.path.join(dataset_path, fish_type)
    
    # fish species folder control
    if os.path.isdir(fish_type_path):
        for subfolder in os.listdir(fish_type_path):
            subfolder_path = os.path.join(fish_type_path, subfolder)
            
            # subfolder control
            if os.path.isdir(subfolder_path) and not subfolder.endswith('GT'):
                for img_name in os.listdir(subfolder_path):
                    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')) :
                        continue
                    
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    
                    if img is None:
                        continue
                    
                    img = cv2.resize(img, (img_width, img_height))
                    img = img / 255.0  # normalization
                    images.append(img)
                    labels.append(fish_type) 

# convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

le = LabelEncoder()
labels = le.fit_transform(labels) # numbered the series alphabetically
y = to_categorical(labels)  # make it categorical (it prints 1 to the class it belongs to)
print(f'Images size: {images.shape}')  # expected size (n, 128, 128, 3)
print(f'Label size: {y.shape}')   # expected size (n, 9) 


In this code block, the images in the dataset are resized to 128x128. The images are normalized for use in model training; this step is important because neural networks work better with values ​​between 0 and 1. The normalized images are added to the images list, while the fish species labels are added to the labels list. Both the images and labels lists are then converted to NumPy arrays. LabelEncoder is used to sort the labels alphabetically and convert them to numerical values. Then, the to_categorical() function converts these numerical labels into a vector representing each class, bringing them to categorical form (for example, [0 0 0 0 0 0 0 0 1] for class number 9).The size of the images determined using the (images.shape) function. The output (n, 128, 128, 3) shows that there are n 128x128 3-channel (RGB) images. n represents the number of samples here, there are 9000 samples in our study.The size of the labels (y.shape) is printed on the screen. The output (n, 9) shows that there are n images and 9 classes. .

In [ ]:
#X = images.reshape(images.shape[0], -1)
#print(X.shape[1])

**X = images.reshape(images.shape[0], -1) Why is it done?**

Machine learning models generally take data in two dimensions; in the form of the number of samples and the number of features. However, since the images are three-dimensional or more, it is necessary to convert each image into a one-dimensional vector. This allows the model to process this data more easily. This transformation is done to put the data in a proper format before entering the neural network.

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten


X = images.reshape(images.shape[0], -1)  # (9000, 49152)

# create training, temporary, and temporary subsets
X_train, X_temporary, y_train, y_temporary = train_test_split(X, y, train_size=0.8)

# create validation and test subsets
X_val, X_test, y_val, y_test = train_test_split(X_temporary, y_temporary, train_size=0.6)


# create model object
model = tf.keras.Sequential()
# create input layer
input_layer = tf.keras.layers.Dense(128, input_shape=(X.shape[1],), activation='relu') 
model.add(input_layer)
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
# add output layer
model.add(tf.keras.layers.Dense(9, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
results = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=128, callbacks=[early_stopping])

# draw train loss
plt.plot(results.history['loss'], label='Train')
# draw validation loss
plt.plot(results.history['val_loss'], label='Test')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()




In this code block, images with dimensions of 128x128 and 3 color channels are flattened and converted into a single line. With this process, an X series of 9000 images is created, where each image consists of 49152 pixels. Then, the data is divided into 80% training and 20% temporary data set. The temporary data set is divided into 60% validation and 40% test data. The divided data sets are carried to the model creation stage. First, an input layer of 128 neurons with an input size of 49152 is used and this layer has an activation function called "relu". Then, 3 layers with 256, 128 and 64 neurons are added and each layer is accompanied by a Dropout layer where 30% of the neurons will be disabled. Finally, an output layer using the "softmax" activation function is added. The reason for using "softmax" is that it is used in multi-class classifications.

Adam optimizer is used for model optimization and the learning rate is set to 0.0001. Categorical_crossentropy is selected as the loss function and accuracy is selected as the performance measurement. Epoch value is set to 50 and batch_size is set to 128 in each epoch. With the early stopping method, if the validation loss does not improve for 5 epochs, training is stopped and the best weights are restored.

Then, the loss values ​​in the training and validation process are visualized by naming the axes as "Epoch" and "Loss". Thanks to this graph, the performance of the model during the training period and its progress in the validation data can be examined.

In [ ]:
# evaluate the performance
test_result = model.test_on_batch(X_test, y_test)
# print the result
print(test_result)

Here, the test_on_batch(X_test, y_test) function evaluates the model on the test datasets X_test and y_test in one go (batch). The metrics of the model such as loss and accuracy are returned.So this code allows us to quickly see the performance of the model on test data.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# evaluate the performance of the model using test data
predictions = model.predict(X)
predicted_classes = np.argmax(predictions, axis=1)  # select classes based on highest probabilities

true_classes = np.argmax(y, axis=1)

print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))

In this code snippet, the classification report and confusion matrix are used to evaluate the performance of the trained model on the test data.
The predict(X) function allows the model to make predictions for all images. This returns an array containing the probability values for each fish species.
Using np.argmax(predictions, axis=1), the highest predicted probabilities are selected and assigned to the predicted_classes array as predicted classes.
Using np.argmax(y, axis=1), the numerical equivalents of the true labels (i.e. the correct classes) are assigned to the true_classes array.
Using classification_report(true_classes, predicted_classes, target_names=le.classes_), a report summarizing the performance of the model is generated. This report contains metrics such as accuracy, recall, and F1 score for each class. The class names are added with target_names.
The confusion_matrix(true_classes, predicted_classes) command creates and prints a confusion matrix that visually represents the model’s correct and incorrect predictions. This matrix shows how accurately each class was predicted and which classes were confused. These steps are used to evaluate the model’s overall success and its performance for each class.